### 1. Setup and Data Loading

In this notebook, we'll integrate the filtered datasets from all supermarkets and perform the final round of cleaning.

In [1]:
import pandas as pd

We begin by loading the cleaned product lists for each supermarket:

In [2]:
pyaterochka = pd.read_csv('filtered_products-2025-03-03-pyaterochka.csv')
lenta = pd.read_csv('filtered_products-2025-03-04-lenta.csv')
winmart = pd.read_csv('filtered_products-2025-03-06-winmart.csv')
coop = pd.read_csv('filtered_products-2025-03-07-coop.csv')

Each product entry includes raw and normalized prices, quantity information (weight, volume, or number of units), its type (e.g., rice, milk, tea) and supermarket.

### 2. Basic Cleaning and Preparation

We drop residual index columns from CSV exports and standardize column names:

In [3]:
pyaterochka = pyaterochka.drop('id', axis=1).rename(columns={'pricing_unit': 'uom'})
lenta = lenta.drop(lenta.columns[0], axis=1).rename(columns={'pricing_unit': 'uom'})
winmart = winmart.drop(winmart.columns[0], axis=1)
coop = coop.drop(coop.columns[0], axis=1)

All product data is combined into a single DataFrame.

In [4]:
all_products = pd.concat([pyaterochka, lenta, winmart, coop], ignore_index=True)

### 3. Manual Fixes and Adjustments

Based on manual inspection, we drop incorrect or irrelevant rows and add any missing items to the dataset.

In [5]:
# drop irrelevant variants that are out of the scope of our analysis
all_products = all_products.drop([684, 224]) # brown rice
all_products = all_products.drop(1877) # wrong type of eggs
all_products = all_products.drop(1683) # oranges without weight
all_products = all_products.drop([1470, 1506]) # not sweetened condensed milk
all_products = all_products.drop([108, 124, 129, 114, 118, 115, 131, 130, 421, 422, 419, 420, 418, 117, 1662]) # drinking yogurt
all_products = all_products.drop([575, 566, 565, 564, 365, 360, 359, 560, 559, 576, 506]) # coffee in drip boxes
all_products = all_products.drop([1294, 1297, 292, 289, 1304, 2310]) # non-white sugar or liquid sugar (2310)
all_products = all_products.drop([443]) # non-wheat spaghetti
all_products = all_products.drop([1499, 1495, 1493, 1498, 1489, 1497, 1478, 202]) # non-rice noodles

# add an eggplant entry that wasn't in the dataset but is present in the supermarket
all_products.loc[10000] = ['Cà tím màng co', 25500, 'kg', 'eggplant', 'Co.op', 1000, 25500, None, None, None, None]

C:\Users\Just Alex\AppData\Local\Temp\ipykernel_1636\2282818764.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_products.loc[10000] = ['Cà tím màng co', 25500, 'kg', 'eggplant', 'Co.op', 1000, 25500, None, None, None, None]


*Note:* These row indices are based on the dataset version processed in this notebook. If any changes are made to the original datasets, the indices may no longer refer to the same items.

Then, we correct issues with price normalization and product details (e.g., missing weight, incorrect volume).

In [6]:
# manually fixing items that failed price normalization (all NaN values)
all_products.loc[1660, 'weight'] = 100
all_products.loc[1736, 'weight'] = 250
all_products.loc[1800, 'volume'] = 500
all_products.loc[1819, 'weight'] = 500

# items that had normalized incorrectly (wrong quantity)
all_products.loc[454, 'weight'] = 300
all_products.loc[1890, 'number_of_units'] = 10
all_products.loc[9, 'weight'] = 220
all_products.loc[1988, 'volume'] = 600 # in reality, this is a pack of 3x200ml
all_products.loc[1714, 'volume'] = 12000 # in reality, this is a pack of 24
all_products.loc[1789, 'volume'] = 600 # in reality, this is a pack of 12
all_products.loc[1792, 'volume'] = 750

# some "весовой" items from Lenta were initially assumed to weigh 1kg, but I just missed the actual weight during extraction.
# fixing those weights here
all_products.loc[598, 'weight'] = 300
all_products.loc[478, 'weight'] = 400
all_products.loc[[446, 453, 456, 459, 460, 461, 462, 463, 464, 467, 468, 469, 476, 488, 599, 728, 731, 733], 'weight'] = 500
all_products.loc[610, 'weight'] = 600
all_products.loc[734, 'weight'] = 700
all_products.loc[448, 'weight'] = 900
all_products.loc[736, 'weight'] = 950
all_products.loc[729, 'weight'] = 1200
all_products.loc[[735, 1162], 'weight'] = 1400
all_products.loc[[458, 1160], 'weight'] = 1500
all_products.loc[730, 'weight'] = 1700
all_products.loc[732, 'weight'] = 1900
all_products.loc[455, 'weight'] = 2700
all_products.loc[451, 'weight'] = 3000

# adding missing weights for some tea products
all_products.loc[[737, 739, 741, 742, 743, 744, 759, 760, 764, 771, 783, 784, 1097, 1003], 'weight'] = 200
all_products.loc[[745, 781, 1053, 1015, 866], 'weight'] = 45
all_products.loc[[746, 747, 752, 755, 756, 762, 763, 765, 767, 769, 779, 780, 782, 785, 1112, 1159, 961], 'weight'] = 50
all_products.loc[[748, 1089], 'weight'] = 180
all_products.loc[[753, 754, 772, 1118, 1149, 1038], 'weight'] = 37.5
all_products.loc[[770, 773, 1024], 'weight'] = 36
all_products.loc[[1471], 'weight'] = 100
all_products.loc[[1133, 986, 898], 'weight'] = 40
all_products.loc[[1123], 'weight'] = 24
all_products.loc[[1107], 'weight'] = 42.5
all_products.loc[[1075], 'weight'] = 27
all_products.loc[[1064], 'weight'] = 150
all_products.loc[[1058], 'weight'] = 34
all_products.loc[[1044], 'weight'] = 170
all_products.loc[[1020], 'weight'] = 170

### 4. Final Price Normalization

Now that all manual quantity adjustments are complete, we recalculate normalized prices to reflect the corrected values.

In [7]:
all_products.loc[all_products['weight'].notna(), 'price_kg'] = all_products['price'] / all_products['weight'] * 1000
all_products.loc[all_products['number_of_units'].notna(), 'price_unit'] = all_products['price'] / all_products['number_of_units']
all_products.loc[all_products['volume'].notna(), 'price_lit'] = all_products['price'] / all_products['volume'] * 1000

Some additional adjustments are needed to fill remaining gaps.
- All milk products must be expressed in liters, so missing volumes are estimated using average density (1.03 g/ml).
- Teas without weight information are not useful for price comparison — a few were manually assigned weights, and the rest are dropped.

In [8]:
# convert milk weight to volume (density ≈ 1.03 g/ml)
all_products.loc[(all_products['product_type'] == 'milk') & (all_products['volume'].isna()), 'volume'] = all_products['weight'] / 1.03
all_products.loc[all_products['volume'].notna(), 'price_lit'] = all_products['price'] / all_products['volume'] * 1000
# drop tea entries without weight (some were manually filled in earlier)
all_products = all_products.loc[~((all_products['product_type'].isin(['black_tea', 'green_tea'])) & (all_products['weight'].isna()))]

### 5. Final Cleanup and Formatting

Each product type will be compared using a single relevant price metric: for example, rice and vegetables by price_kg, water by price_lit, and eggs by price_unit. To avoid confusion in later analysis, we now remove irrelevant values from quantity-related columns — e.g., clearing weight and price_kg for liquids, or number_of_units for buckwheat.

We also create a new DataFrame for the cleaned version.

In [9]:
clean_products = all_products

# correspond product types and metrics 
unit_types = ['egg']
volume_types = ['water', 'sunflower_oil', 'soybean_oil', 'milk', 'fish_sauce']
weight_items = ~clean_products['product_type'].isin(unit_types + volume_types)

# set None to irrelevant fields based on product_type
clean_products.loc[clean_products['product_type'].isin(unit_types), ['weight', 'price_kg', 'volume', 'price_lit']] = None
clean_products.loc[clean_products['product_type'].isin(volume_types), ['weight', 'price_kg', 'number_of_units', 'price_unit']] = None
clean_products.loc[weight_items, ['volume', 'price_lit', 'number_of_units', 'price_unit']] = None

Next, we convert all prices to USD for consistent comparison. Since exchange rates can fluctuate — especially the ruble — we use the average for the month preceding data collection (February 7 to March 7).
(Source: *exchange-rates.org*)

In [10]:
# save the dataset with prices in local currencies as a backup
clean_products.to_csv(f'clean_products-2025-03-12-local.csv')

rub_to_usd = 0.011056
vnd_to_usd = 0.000039214

clean_products.loc[clean_products['supermarket'].isin(['Pyaterochka', 'Lenta']), ['price','price_kg', 'price_unit', 'price_lit']] *= rub_to_usd
clean_products.loc[clean_products['supermarket'].isin(['Winmart', 'Co.op']), ['price','price_kg', 'price_unit', 'price_lit']] *= vnd_to_usd

Finally, we save the cleaned dataset to a new file.

In [11]:
clean_products.to_csv(f'clean_products-2025-03-12-usd.csv')